In [ ]:
#install scikit-image on computer if you have not already
'''
pip install scikit-image
'''

In [ ]:
import numpy as np
import random
from skimage import io, measure, feature
from PIL import Image, ImageDraw

In [ ]:
#scikit-image has packages tifffile, pillow, networkx, lazy-loader, imageio, and scikit-image
#scikit-image, aka skimage
#python pillow= PIL

In [ ]:
#Use ImageJ to process your sample image
#Outline cell border. Click Analyze> Tools> ROI Manager. Add cell border to ROI Manager. Click More> Save. Save ROI
#Click Analyze> Tools> Save XY Coordinates. This will save all the coordinates within the cell border in a .csv file
#this will save a .csv spreadsheet of all the coordinates WITHIN the cell border

In [ ]:
def spotscrambler(file, cellmask):
    with open(file, "rb") as image:
        #load binary image from tif file
        binary=io.imread(image)
       
        #label connected regions in image
        #note: default is full connectivity (in this case, 2). can change to connectivity =1 or 2
        #connectivity 1 is grouping square pixels that share an edge
        #connectivity 2 is grouping square pizels that share an edge or corner
        labeled=measure.label(binary, connectivity=1)
        
        #extract properties (area, centroid, coordinates, label, etc) of each labeled region
        regions=measure.regionprops(labeled)

        #turn regions/spots into circles with the same area, and then get radius in a list
        #drawing circles with square pixels means radius is in pixels, which means only integers of pixels
        radii=[]
        for spot in regions:
            radius=int(np.sqrt(spot.area/np.pi))
            radii.append(radius)
        
        #sort radii list from largest to smallest
        radii.sort(reverse=True)
        
        #print if you want to see all the radii after rounding down to nearest integer
        '''
        print(radii))
        '''
        
        #in earlier step, when making radii into integers, some radii less than 1 were turned into zeros
        #if radii= 1, circle will be 5 pixels in area.
        #smallest possible "circle" is made of 1 pixel. therefore smallest possible radius is 0.5.
        #turn zeros into 0.5 to keep same number of spots
        for x in range(len(radii)):
            if radii[x]==0:
                radii[x]=0.5

        #print to check that radii of zero length were turned into 0.5
        '''
        print(radii)
        '''
        
        #print if you want to know how many spots were detected
        '''
        print(f"The length of radii list is {len(radii)}.")
        '''
        
     
    #acquire coordinates within cell perimeter after ImageJ processing. open spreadsheet
    fillcoords=[]
    with open(cellmask, "r") as mask:
        csvfile= mask.readlines()[1:]
        #csvfile is now a list of strings(lines)
        for string in csvfile:
            #change strings into list of x and y, not third column in csvfile.
            list=string.split(",")[0:2]
            #x and y are currently strings of numbers. change into float.
            for i in range(len(list)):
                list[i]=float(list[i])
            #append each list of x, y floats to fillcoords list
            fillcoords.append(list)

    #create a new image with size of desired pixels
        #binary.shape gives dimensions (pixels) of binary image- height by width
        image_size=(binary.shape[1], binary.shape[0])
        #Image.new makes images with dimensions width by height
        new_image=Image.new('L', image_size, (0))

    #iterate through list of radii, pick random coordinate for circle centers
    #check if new circle centers will overlap with existing circles
    new_center_coords=[]
    #generate center coordinate for first/largest circle
    tentativecenter=random.choice(fillcoords)
    new_center_coords.append(tentativecenter)
    #now generate center coordinates for all the rest of the circles
    for i in range(1,len(radii)):
        #tentativecenter needs to exist outside of while loop. 
        #if tentativecenter only in while loop, tentativecenter doesn't exist once you break out of while loop.
        #assuming overlap
        overlap=True
        while overlap==True:
            overlapcount=0
            tentativecenter=random.choice(fillcoords)
            #checking if new circle center overlaps with any existing circles
            for j,[x,y] in enumerate(new_center_coords):
                distance= (np.sqrt(((tentativecenter[0] - x)**2) + ((tentativecenter[1] - y)**2)))
                shortestdistance= radii[i]+radii[j]
                if distance >= shortestdistance:
                    continue
                if distance < shortestdistance:
                    #overlaps
                    overlapcount=overlapcount+1
                    overlap=True
                    #break out of for loop
                    break  
            if overlapcount==0:
                overlap=False
                #exits while loop
            if overlapcount>0:
                overlap=True
                #goes back to top of while loop
        new_center_coords.append(tentativecenter)
    #print if you want to know how many new circles will be generated. should match the number of original spots
    '''            
    print(f"The length of new_center_coords is {len(new_center_coords)}")
    '''

    #draw circles at new coordinates
    circlecount=0
    draw= ImageDraw.Draw(new_image)
    for i, [x,y] in enumerate(new_center_coords):
        if radii[i] != 0:
            draw.circle([x,y],radii[i], fill=255)
            circlecount=circlecount+1
            
    #print if you want to know how many circles were drawn. should match number of center coordinates
    '''
    print(circlecount)
    '''

    #save new image as tif file
    new_image.save(f"{file}.SCRAMBLE.tif")

In [ ]:
import os
imagedirectory= "pathtoimagefolder"
csvdirectory= "pathtocsvfolder"

In [ ]:
#images and csv files can be in different folders or same folder. doesn't matter.
imagelist=[]
csvlist=[]
for name in os.listdir(imagedirectory):
    if name.endswith(".tif"):
        imagelist.append(f"{imagedirectory}/{name}")
for name in os.listdir(csvdirectory):
    if name.endswith(".csv"):
        csvlist.append(f"{csvdirectory}/{name}")
#sort your imagelist and csvlist so that they pair correctly
imagelist.sort()
csvlist.sort()

#print to check that all your files/order are correct
'''
print(imagelist)
print(csvlist)
'''

#print to check that length of both lists are the same
'''
print(len(imagelist))
print(len(csvlist))
'''

In [ ]:
#to use SpotScrambler on whole folders
for i in range(len(imagelist)):
    spotscrambler(imagelist[i], csvlist[i])

In [ ]:
#to use SpotScrambler one image at a time
#function takes two inputs. first input is binary image of spots. second input is .csv file of cell surface area

spotscrambler("pathtobinaryimage","pathtocsvfile")